### Match administrative boundaries 
This code automatically links administrative boundaries based on a spatial location <br>
Last modified : August 25 2021 by Imryoung Jeong (neptune0118@gmail.com)


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',100)
pd.set_option('display.float_format', '{:.4f}'.format)

###### Functions
----

In [1]:
def admin_match(data_to_match, data_base):
    
    result = gpd.overlay(data_to_match, data_base, how='intersection')
    result = result.sort_values([idx_base], ascending = True)
    result['match_rate'] = (result['geometry'].area * 10**4) / result['area']
    
    return result

# Case 1. Based on the threshold (e.g. intersect more than NN%)
def choose_share(data):
      
    filtered_data = data['match_rate'] > FRAC
    result = data[filtered_data].reset_index()
    
    print(len(result))
    
    return result

# Case 2. Select a region with the largest share
def choose_max(data):

    idx = data.groupby(by=idx_match)['match_rate'].transform(max) == data['match_rate']
    result = data[idx]
    
    result.drop(columns=['geometry'], inplace=True)
    print(len(result))
    
    return result

---

In [2]:
idx_base = 'id'
idx_match = 'adm_dr_cd'
FRAC = 0.1

# Import shapefiles used
dir_base = "/Users/imryoung/Dropbox/EconAI_Maps/Data/cookie_mold/Population_grid/population_grid_combined_2km.shp"
base = gpd.read_file(dir_base,
                     encoding = 'euc-kr').set_crs(epsg=5179).to_crs(epsg=4326)
base = base[['id','geometry','r2_val_r']]

dir_match = "/Users/imryoung/Dropbox/EconAI_Imryoung/GIS/2016_ngii_town/nlsp_003001001.shp"
match = gpd.read_file(dir_match,
                      encoding = 'euc-kr').to_crs(epsg = 4326)
match.columns = match.columns.str.lower()

,id,geometry,r2_val_r
0,1,"POLYGON ((124.60796 37.98245, 124.63071 37.983...",60
1,2,"POLYGON ((124.60867 37.96445, 124.63141 37.965...",0
2,3,"POLYGON ((124.60938 37.94645, 124.63211 37.947...",0


In [4]:

m2 = match[['gid','geometry','val']].copy()
m2['area'] = match.area * 10**4

result = admin_match(m2, base)
#result = choose_share(result)

result.head(3)

<ipython-input-4-4d6d31559ec6>:7: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m2['area'] = match.area * 10**4
<ipython-input-1-5fcd50756ce3>:14: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  result['matching_rate'] = (result['geometry'].area * 10**6) / result['area']


,gid,val,area,id,r2_val_r,geometry,matching_rate
4344,28720330,5491.0000,52.5248,1,60,"POLYGON ((124.61181 37.96453, 124.61211 37.965...",3.9940
4345,28720330,5491.0000,52.5248,2,0,"POLYGON ((124.62743 37.94689, 124.62738 37.947...",3.6130
4346,28720330,5491.0000,52.5248,3,0,"POLYGON ((124.63252 37.93652, 124.63247 37.936...",0.6157
